In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mybiotools as mbt
import pr_peaks
import pysam

# 2018-02-27 Tetramer boost

I still don't want to start thinking of super-weird network topologies that might explain the non-monotonic h-enhancement. One other thing I might want to consider is that there are cases in which there is an abnormal stability enhancement due to the fact that the searchers interact between them and gain stability. I'm thinking specifically in the formation of DNA-mediated PR tetramer loops. The idea would be that if only a few sites are occupied at any given time, there is little probability of formation of the super-loops. Imagine the case in which there is only one searcher: then there is no possibility of formation of a di-molecular complex.

Here I'm thinking that the searcher in question is a PR homo-dimer. The PR as a monomer practically does not exist, and there is very little possibility of formation of tetramers in free solution.

From the techical point of view the simulation code now is slightly more involved. I need to figure out a way of including the contacts between the sites. I'll suppose a contact matrix between the sites.

I'll drop the transition matrix non-uniformness, for simplicity. I'll also drop the "searcher" class, which seems not important. I'll also drop the "sigma" parameter which is totally irrelevant.

In [ ]:
def init_omega_t(n,mu,seed=None) :
    # init the random number generator if it was passed
    if seed is not None :
        np.random.seed(seed)
    # init omega vector
    omega_t = np.zeros(n,dtype=bool)
    # fill with initial occupancy
    omega_t[np.random.choice(n,mu,replace=False)] = True
    return omega_t

In [ ]:
def run_chair_simulation(nsteps,omega_t_initial,H,site_taus,boost,
                         seed=None,teq=0,tsample=1) :
    # init the random number generator if it was passed
    if seed is not None :
        np.random.seed(seed)
    # make an internal copy of the initial omega_t vector
    omega_t = omega_t_initial.copy()
    # no need to pass N,n,m through the arguments of the function
    n = omega_t.size
    m = omega_t.sum()
    # init jumping matrix
    J = np.zeros((n,n)).astype(np.int32)
    # init searcher_sites
    searcher_sites = -1 * np.ones(n,dtype=np.int32)
    searcher_sites[omega_t] = np.arange(m)
    # init searcher_times
    searcher_times = np.zeros(m)
    for s in xrange(m) :
        searcher_id = searcher_sites[s]
        if searcher_id != -1 :
            searcher_times[searcher_id] = site_taus[s]
    # init sampling matrix
    nsamples = (nsteps-teq)/tsample
    samples = np.zeros((nsamples,n),dtype=bool)
    i_sample = 0
    # cycle on time
    for step in xrange(1,nsteps+1) :
        # cycle on the searchers
        for s in xrange(n) :
            searcher_id = searcher_sites[s]
            if searcher_id != -1 :
                # if the searcher has to stay longer on the site, skip it
                if step>=searcher_times[searcher_id] :
                    # now get the next site
                    next_site = np.random.choice(np.where(~omega_t)[0])
                    # update omega matrix
                    omega_t[s] = False
                    omega_t[next_site] = True
                    # update jumping matrix
                    J[s,next_site] += 1
                    # update searcher_sites
                    searcher_sites[s] = -1
                    searcher_sites[next_site] = searcher_id
                    # update searcher_times
                    next_td = site_taus[next_site]
                    for contact in H[next_site] :
                        contact_searcher_id = searcher_sites[contact]
                        if contact_searcher_id != -1 :
                            next_td *= boost
                            searcher_times[contact_searcher_id] = step + next_td
                            break
                    searcher_times[searcher_id] = step + next_td
        # update samples
        if step>teq and (step-teq)%tsample==0 :
            samples[i_sample,:] = omega_t
            i_sample += 1
    return omega_t, J, samples

In [ ]:
class JumpingModel :
    def __init__ (self,H,site_taus,boost) :
        self.H = H
        self.site_taus = site_taus
        self.boost = boost
        self.J = {}
        self.samples = {}
        self.theta = {}
    def run(self,nsteps,omega_t_initial,seed=None,teq=0,tsample=1) :
        omega_t,self.J[mu],self.samples[mu] = \
            run_chair_simulation(nsteps,
                                 omega_t_initial,
                                 self.H,
                                 self.site_taus,
                                 self.boost,
                                 seed=None,teq=0,tsample=1)
        self.theta[mu] = self.samples[mu].sum(axis=0)/float(self.samples[mu].sum())

This is the simulation code. Now I'll test what happens if everything is normal.

In [ ]:
# general simulation parameters
nsteps = 10000
n = 100
H = [[] for i in xrange(n)]
boost = None
# init site_taus
Hsites = [2,6]
Lsites = [i for i in xrange(n) if i not in Hsites]
site_taus = 2.0*np.ones(n)
site_taus[Hsites] = 20.0
# init the Jumping Model
uniform = JumpingModel(H,site_taus,boost)
# cycle on mu values
mus = np.arange(1,20,2)
np.random.seed(85498)
samples = {}
for mu in mus :
    mbt.log_message('Uniform','mu = %d'%(mu))
    # init omega_t
    omega_t_initial = init_omega_t(n,mu)
    uniform.run(nsteps,omega_t_initial)
pr_peaks.H_to_L(uniform,Hsites,Lsites)

In [ ]:
plt.plot(mus,uniform.H_to_L,'o--')
plt.xlabel(r'$\mu$')
plt.ylabel('H to L ratio')
plt.show()

Okay, this is the normal result that we expect. Now let's proceed with a simple case: an H site is in contact with another H site.

In [ ]:
# general simulation parameters
nsteps = 10000
n = 100
# init site_taus
Hsites = [2,6,10,40,50,60,70,80]
Lsites = [i for i in xrange(n) if i not in Hsites]
site_taus = 2.0*np.ones(n)
site_taus[Hsites] = 20.0
# init H and boost
H = [[] for i in xrange(n)]
H[Hsites[0]] = [Hsites[1]]
H[Hsites[1]] = [Hsites[0]]
boost = 4.0
# init the Jumping Model
HH = JumpingModel(H,site_taus,boost)
# cycle on mu values
mus = np.arange(1,20,2)
np.random.seed(85498)
samples = {}
for mu in mus :
    mbt.log_message('HH','mu = %d'%(mu))
    # init omega_t
    omega_t_initial = init_omega_t(n,mu)
    HH.run(nsteps,omega_t_initial)
pr_peaks.H_to_L(HH,Hsites,Lsites)

In [ ]:
plt.plot(mus,HH.H_to_L,'o--',color='b',label='HH model')
plt.plot(mus,uniform.H_to_L,'o--',color='r',label='Uniform model')
plt.xlabel(r'$\mu$')
plt.ylabel('H to L ratio')
plt.legend(loc='upper right')
plt.show()

So I finally obtain something which is convincingly pointing to the direction that this can be an explanation for the h-enhancement!

## Back to the data

I want now to look at potential ways to pin down the parameters of the model based on the data that we have.

I want to look at the matrix of contacts between H and H peaks. The first approach I want to take is to look at the BAM file of the HiC experiment and look at the number of reads corresponding to the contacts between the H and the H sites.

In [ ]:
# load data corresponding to the various conditions
high       = pr_peaks.Condition('high'  ,'all_treated',0.05,'gv_107_01_01_chipseq')
medium     = pr_peaks.Condition('medium','3HCP'       ,0.50,'gv_109_01_01_chipseq')
low        = pr_peaks.Condition('low'   ,'1HCP'       ,10.0,'gv_111_01_01_chipseq')

In [ ]:
# extract the peak information
Hpeaks = high.peaks
Mpeaks = medium.peaks
Lpeaks = low.peaks

In [ ]:
# get the location of the BAM files corresponding to the HiC and HiChIP data
hichip_sample_id = '9a7c4a68d_6313677b5'
hic_sample_id = '9a7c4a68d_51720e9cf'
hichip_file = mbt.hic_bam_location(hichip_sample_id)
hic_file = mbt.hic_bam_location(hic_sample_id)

Now I write a piece of code to do this dirty business.

In [ ]:
def peak_id_string(peak) :
    return '%s_%d_%d'%(peak['chr'],peak['start'],peak['end'])

In [ ]:
def peak_contacts_from_bam(bam,regions1,regions2,flag=1807) :
    """
    Using pysam (that is, samtools), take a bam file and extract the reads of a
    hi-c file corresponding to the contacts between the regions defined in the 
    variables `regions1` and `regions2`.
    """
    chromosomes = np.unique(regions2['chr'])
    regions2_ordered = {}
    for chromosome in chromosomes :
        regions2_ordered[chromosome] = np.array([p for p in regions2 if p['chr']==chromosome])
        regions2_ordered[chromosome].sort()
    # make a table of indices of the regions to evaluate
    regions2_indices = {}
    i = 0
    for region2 in regions2 :
        id_string = peak_id_string(region2)
        regions2_indices[id_string] = i
        i+=1
    # init the matrix
    n1 = len(regions1)
    n2 = len(regions2)
    H = np.zeros((n1,n2),dtype=np.int32)
    # the 'b' flag here indicates that we are dealing with a bam file
    with pysam.AlignmentFile(bam,'rb') as samfile :
        # iterate over all the reads in regions1
        for i,region1 in enumerate(regions1) :
            chromosome,start1,end1 = region1
            chromosome = str(chromosome)
            samiter = samfile.fetch(chromosome,start1-2500,end1+2500)
            mbt.log_message('peak_contacts_from_bam','Read %d/%d'%(i,n1))
            # iterate
            for read in samiter :
                if read.flag <= flag :
                    pos2 = read.next_reference_start
                    chromosome2 = read.next_reference_name
                    if chromosome2 in chromosomes :
                        for region2 in regions2_ordered[chromosome2] :
                            c,start2,end2 = region2
                            if pos2>=start2-2500 and pos2<=end2+2500:
                                id_string = peak_id_string(region2)
                                j = regions2_indices[id_string]
                                H[i,j] += 1
                                H[j,i] += 1
    return H

In [ ]:
H = peak_contacts_from_bam(hic_file,Hpeaks,Hpeaks)

In [ ]:
Hnodiag = H.copy()
for i in xrange(H.shape[0]) :
    Hnodiag[i,i] = 0
fig = plt.figure(figsize=(8,8))
ax = plt.subplot(111)
ax.matshow(1-np.log(Hnodiag),cmap=plt.cm.gray)
plt.show()

It is pretty clear that there is nothing here. The regions that correspond to the regions under examination are too small to give a significant number of reads outside of the diagonal. Better to look at the binned HiC matrix and extract the columns/rows corresponding to the peaks.

In [ ]:
resolution = 50000
hic_mat_file = mbt.hic_location(hic_sample_id,resolution)
Hic = mbt.parse_hic(hic_mat_file)

In [ ]:
chromosomes = np.unique(Hic['chr'])
chromosomes.sort()
for chromosome in chromosomes :
    Hic_chromosome = np.array([r for r in Hic if r['chr']==chromosome])
    ivals = np.unique(Hic_chromosome['i'])
    print "Chromosome %s: %4d  values"%(chromosome,ivals.size)

In [ ]:
def counts_to_hic (counts,start,end,resolution) :
    """
    Returns a complete filled matrix given the 'counts' array, by taking
    for granted that the counts correspond to a given chromosome.
    """
    N = (end-start)/resolution + 1
    H = np.zeros((N,N),dtype=counts['val'].dtype)
    for h in counts :
        i = (h['i']-start)/resolution
        j = (h['j']-start)/resolution
        H[i,j] = H[j,i] = h['val']
    return H

In [ ]:
chromosome = 'chr20'
Hpeaks_chromosome = np.array([p for p in Hpeaks if p['chr']==chromosome])
hic_chromosome = np.array([h for h in Hic if h['chr']==chromosome])
N = max(hic_chromosome['i'].max(),hic_chromosome['j'].max())
n = N/resolution
H = counts_to_hic(hic_chromosome,0,N,resolution)

In [ ]:
x = np.arange(n)
y = np.zeros(n)
for p in Hpeaks_chromosome :
    i = p['start']/resolution
    y[i] = 1.0

In [ ]:
fig = plt.figure(figsize=(10,11))
gs = plt.GridSpec(2,1,hspace=0,height_ratios=[10,1])
ax = plt.subplot(gs[0,0])
ax.matshow(1-np.log(H),cmap=plt.cm.Greens)
ax = plt.subplot(gs[1,0],sharex=ax)
mbt.line_plot(ax,x,y)
plt.show()

Now, there is no clear pattern emerging from this figure. This is a case in which it is difficult to draw conclusions just by visually looking at the patterns of the HiC data. Another approach is possible though. In a scenario in which the tetramerization is really the responsible for the h-enhancement, there needs to be a subset of the H peaks that show the enrichment. I'll show this in the next notebook.